# 031_欠損値を複数の参照dfを参照して穴埋めする関数を作成:　試作版

## 目的

- 更新用データフレームの欠損値を別にある参照用データフレームの値で穴埋めしたい

## 条件

- 使用する情報は、データフレームは更新用（tbl）と参照用データフレームが２つ(emst、fmst)、priceの初期値（固定値）
- tbl.priceが欠損値の時に以下の条件で穴埋め処理を行う

### ルール１
- tbl.lcdがEで始まる時は、emstのpriceで埋めます
- tbl.lcdがE始まりではないときは、fmstのpriceで埋めます

### ルール２
- emst.lcd、fmst.lcdにtbl.mstと同じlcdがない場合があります。
- emst.lcd、fmst.lcdにtbl.mstと同じlcdがあっても該当するpriceの値が入っていない場合があります。

### ルール３
- 上のルールで価が決まらない場合には、予め決めて置いた固定値でで埋めてしまいます


## 1. 入力：データフレームを作成

In [1]:
import pandas as pd


# 参照df その１
emst = pd.DataFrame(
    [
    {"cd":"E001","val":1},
    {"cd":"E002","val":3}
    ],columns=["lcd","val"]
)
# 参照df その２
fmst = pd.DataFrame(
    [
    {"cd":"F001","val":2},
    {"cd":"F002","val":4}
    ],columns=["lcd","val"]
)

# 更新tbl
def create_tbl():
    tbl = pd.DataFrame(
        [
        {"cd": "E001",  "val": 100},
        {"cd": "E001",  "val": ''},
        {"cd": "E001",  "val": None},
        {"cd": "E002",  "val": ''},
        {"cd": "F001",  "val": 100},
        {"cd": "F001",  "val": ''},
        {"cd": "F001",  "val": None},
        {"cd": "F002",  "val": ''}
        ],columns=["lcd", "val"]
    )
    return tbl

tbl = create_tbl()
tbl


,lcd,val
0,NaN,100
1,NaN,
2,NaN,None
3,NaN,
4,NaN,100
5,NaN,
6,NaN,None
7,NaN,


### 出力：理想形

## 参考：for文による処理

In [18]:
import math

def fillnan_mst_for(tbl, fmst, emst):
    """
    func
    tblのvalがNanのとき、
    cdが'F'で始まる時は、fmstのvalで補完埋めします
    cdが'E'で始まる時は、emstのvalで補完埋めします
    それ以外のときはNanのままにします。
    In
    tbl : columns=["cd","val"]のpd.DataFrame（穴埋め更新対象）
    fmst : columns=["cd","val"]のpd.DataFrame（参照用）
    emst : columns=["cd","val"]のpd.DataFrame（参照用）
    out
    fmst, emstを参照してNanを補完埋めしたtbl
    """
    for i in range (len(tbl)):
        if math.isnan(tbl.loc[i, "val"]):
            if tbl.loc[i, "cd"].startswith("F"):
                tbl.loc[i, "val"] = fmst[fmst["cd"] == tbl.loc[i, "cd"]]["val"].values[0]
            elif tbl.loc[i, "cd"].startswith("E"):
                tbl.loc[i, "val"] = emst[emst["cd"] == tbl.loc[i, "cd"]]["val"].values[0]
        
    return tbl

In [ ]:
# df作成
tbl = create_tbl()
# 空文字をNoneに置換
tbl = tbl.replace({'': None})

%time tbl = fillnan_mst_for(tbl, fmst, emst)

